In [ ]:
!pip install -U jina==2.2.5
!pip install torch==1.9.0
!pip install transformers==4.5.1
!pip install sentence_transformers
!pip install gradio


  Using cached torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
  Using cached transformers-4.5.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
ERROR: pip's dependency resolver does not currently take into account all the pack

In [ ]:

from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np 
import torch
import warnings
#warnings.filterwarnings('ignore')

In [ ]:
import shutil
shutil.move("/content/gdrive/MyDrive/workspace", "/content")

'/content/workspace'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v1')

In [ ]:

model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model,'device':'cpu'}
        
    )
    .add(
        name="text_indexer",
        uses='jinahub://SimpleIndexer',
        #uses_with={'device': 'cuda'},
        
    )
)

In [ ]:
def query_text(Text):
    query = Document(text = Text)
    print()
    with flow:
        response = flow.search(inputs = query,parameters={'limit':20},uses_with ={'device':'cpu'},gpus='all' ,return_results = True)
    matches = response[0].docs[0].matches

    match=matches[0]
    text=match.text
    show=match.tags['show']
    season = match.tags['season']
    episode = match.tags['episode']
    speaker = match.tags['speaker']
    appears_on= show +" "+ season +" "+ episode
    if type(speaker)==float:
        speaker="Not available"
    return text,speaker,appears_on,type(speaker)
        

In [ ]:
query = Document(text = "Today is not the day i die")
print()
with flow:
    response = flow.search(inputs = query,parameters={'limit':20},uses_with ={'device':'cpu'} ,return_results = True)

Output()

           Flow@1233[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:47923
	🔒 Private network:	172.28.0.2:47923
	🌐 Public address:	35.227.117.45:47923
   text_encoder@1380[E]:TypeError('`traversal_paths` needs to be `Sequence[str]`')
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 285, in _msg_callback
    processed_msg = self._callback(msg)
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 271, in _callback
    msg = self._post_hook(self._handle(self._pre_hook(msg)))
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 226, in _handle
    peapod_name=self.name,
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/request_handlers/data_request_handler.py", line 165, in handle
    field='groundtruths',
  File "/usr/local/lib/python3.7/dist-packages/ji

In [ ]:
query = Document(text = input("Enter the Text - "))
print()
with flow:
    response = flow.search(inputs = query,parameters={'limit':20},uses_with ={'device':'cpu'},gpus='all' ,return_results = True)
matches = response[0].docs[0].matches

for i in matches[:5]:
    print()
    print(str(i.text),"\n")
    print("series :", i.tags['show'],"\n")
    print("season :", i.tags['season'],"\n")
    print("episode :", i.tags['episode'],"\n")
    print("speaker :", i.tags['speaker'],"\n")
    
    print("_________________________________________________________________________________________________________________-\n")

Enter the Text - today is not the day i die

⠦ 1/3 waiting text_encoder text_indexer to be ready...   text_encoder@783[C]:can not load the executor from /root/.jina/hub-packages/u9pqs8eb/config.yml
   text_encoder@783[E]:ExecutorFailToLoad() during <class 'jina.peapods.runtimes.zmq.zed.ZEDRuntime'> initialization
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/request_handlers/data_request_handler.py", line 78, in _load_executor
    extra_search_paths=self.args.extra_search_paths,
  File "/usr/local/lib/python3.7/dist-packages/jina/jaml/__init__.py", line 613, in load_config
    return JAML.load(tag_yml, substitute=False)
  File "/usr/local/lib/python3.7/dist-packages/jina/jaml/__init__.py", line 90, in load
    r = yaml.load(stream, Loader=JinaLoader)
  File "/usr/local/lib/python3.7/dist-packages/yaml/__init__.py", line 81, in load
    return loader.get_single_data()
  File 

RuntimeFailToStart: ignored

In [ ]:
import gradio as gr


In [ ]:

face = gr.Interface(fn=query_text, 
                    inputs="text", outputs=[gr.outputs.Label(type="auto", label="The actual Script"),
                                            gr.outputs.Label(type="auto", label="Spoken by - "),
                                            gr.outputs.Label(type="auto", label="Appears On - ")],
                     layout='vertical',
                    title="TV Shows - Neural Search",
                    description="""Enter your Dialouge from any TV shows and retrive the details of the script on the speaker, 
                                  Episode,Season and name of the TV show\n\n. Currently F.R.I.E.N.D.S, How I met your Mother, Two and half-men are updated""",
                    theme="grass"
                    )
face.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://39392.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://39392.gradio.app')